In [41]:
import ee
import geemap
import geopandas 
from ipyleaflet import GeoJSON
from zipfile import ZipFile
ee.Initialize()

In [42]:
#file_name = "/data/MGN2021_DPTO_POLITICO.zip" 
#print("Showing content files... \n")
#with ZipFile(file_name, 'r') as departamentos:
    # printing all the contents of the zip file
#    departamentos.printdir()
#print('---------- \n')





In [45]:
image = (ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CLOUD").
         select("cloud_fraction").
         filterDate('2022-12-20', '2022-12-21'))

Map = geemap.Map()

In [50]:
departamentos_shp = '/home/bojack/Documents/Tropomi/HIDES_repo/GEE_Colombia/data/MGN_MPIO_POLITICO.shp'
departamentos = geemap.shp_to_ee(departamentos_shp)


'utf-8' codec can't decode byte 0xcd in position 6: invalid continuation byte


In [ ]:
band_viz = {
    'min': 0,
    'max': 0.95,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

#### ee.ImageCollection.mean
Reduces an image collection by calculating the mean of all values at each pixel across the stack of all matching bands. Bands are matched by name.

In [ ]:
#Map.addLayer(image.mean(), band_viz, 'S5P Cloud')
Map.setCenter(-74.297333, 4.570868, 5);

# Se define la region de interes
xMin = -65.654297
yMin = -4.954507
xMax = -82.573242
yMax = 13.330474
region = ee.Geometry.Rectangle([
    [xMin, yMin],
    [xMax, yMax]
  ])

In [ ]:
# se obtiene las coordenadas de la región dibujndo la region con un poligono
#region = Map.user_roi #.getInfo()
#print(region)


In [ ]:
#Map.addLayer(image.mean(), band_viz, 'S5P Cloud')
#geemap.clip_image(Map, region, Map)


El resultado de dibujar el poligono y obtener las coordenadas es un json:

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -82.573242,
              -4.954507
            ],
            [
              -82.573242,
              13.330474
            ],
            [
              -65.654297,
              13.330474
            ],
            [
              -65.654297,
              -4.954507
            ],
            [
              -82.573242,
              -4.954507
            ]
          ]
        ]
      }
    }
  }
})


In [ ]:
# Mosaic the visualization layers and display (or export).
mosaic = image.mosaic()
# Map.addLayer(mosaic, {}, 'mosaic', False)

# Create a circle by drawing a 20000 meter buffer around a point.
clipped = mosaic.clip(region)

# Display a clipped version of the mosaic.
Map.addLayer(clipped, band_viz, 'Clipped image')

# Display the map
Map